In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

from pathlib import Path

import keras_tuner as kt

from importlib import reload
import hyper_model_creator
import keras_model_creator
import sentinel_utils

In [2]:
trials_dir = Path('trials', 'hyperband_resnet')
trial_metric = 'val_weightedf2score'

In [3]:
callbacks = [
    tf.keras.callbacks.TensorBoard(trials_dir.joinpath('board')),
    tf.keras.callbacks.EarlyStopping(
        monitor=trial_metric, patience=20, mode='max'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor=trial_metric, factor=0.5, patience=8, min_lr=1e-6,
    ),
]

In [ ]:
reload(hyper_model_creator)

hypermodel = hyper_model_creator.BuildHyperModel()

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=trials_dir.parent,
    project_name=trials_dir.name,
    hyperband_iterations=3,
    max_epochs=100,
    overwrite=False,
)
tuner.search(
    verbose=1, 
    callbacks=callbacks
)

Reloading Tuner from trials/hyperband_resnet/tuner0.json

Search: Running Trial #435

Value             |Best Value So Far |Hyperparameter
True              |True              |class_weight
2017              |2017_2018_2019    |training_years
l2                |l1l2              |kernel_regularizer
0.5               |0.5               |spatial_dropout
leaky_relu        |leaky_relu        |activation
2                 |4                 |pool_size
0.1               |0.1               |dropout
True              |False             |bias_initializer
0.001             |0.01              |learning_rate
binary_crossent...|binary_crossent...|loss_function
12                |34                |tuner/epochs
4                 |12                |tuner/initial_epoch
3                 |4                 |tuner/bracket
1                 |3                 |tuner/round
0420              |0131              |tuner/trial_id



/home/pj/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 102 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 5/12


I0000 00:00:1724185796.404239    5499 service.cc:145] XLA service 0x7fe3bc02d480 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724185796.404302    5499 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9


   1/3550 ━━━━━━━━━━━━━━━━━━━━ 14:39:10 15s/step - accuracy: 0.2969 - auc: 0.8108 - loss: 0.6676 - prc: 0.6119 - precision: 0.6447 - recall: 0.4118 - weightedf1score: 0.4315 - weightedf2score: 0.4174

I0000 00:00:1724185807.419946    5499 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3550/3550 ━━━━━━━━━━━━━━━━━━━━ 247s 65ms/step - accuracy: 0.3847 - auc: 0.8350 - loss: 0.6127 - prc: 0.6289 - precision: 0.6749 - recall: 0.4481 - weightedf1score: 0.4691 - weightedf2score: 0.4557 - val_accuracy: 0.3478 - val_auc: 0.8332 - val_loss: 0.5809 - val_prc: 0.6177 - val_precision: 0.6635 - val_recall: 0.4370 - val_weightedf1score: 0.4667 - val_weightedf2score: 0.4470 - learning_rate: 0.0010
Epoch 6/12
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 227s 64ms/step - accuracy: 0.3927 - auc: 0.8410 - loss: 0.5612 - prc: 0.6405 - precision: 0.6850 - recall: 0.4511 - weightedf1score: 0.4781 - weightedf2score: 0.4607 - val_accuracy: 0.4011 - val_auc: 0.8423 - val_loss: 0.5464 - val_prc: 0.6570 - val_precision: 0.7215 - val_recall: 0.4346 - val_weightedf1score: 0.4619 - val_weightedf2score: 0.4438 - learning_rate: 0.0010
Epoch 7/12
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 233s 66ms/step - accuracy: 0.3941 - auc: 0.8437 - loss: 0.5332 - prc: 0.6447 - precision: 0.6877 - recall: 0.4535 - weightedf1score: 0.4811

In [ ]:
# best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

# best_hp.values['units'] = 128
# best_hp.values['dropout'] = 0.3

# model = tuner.hypermodel.build(best_hp)
# model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

In [ ]:
# tuner.results_summary(num_trials=2)

In [ ]:
# %load_ext tensorboard
# tb_path = str(trials_dir.joinpath('board'))
# %tensorboard --logdir "$tb_path"